# Inspection

In [ ]:
import pandas as pd
import joblib
from pprint import pprint

# 1) load CV table and show top configs
cv = pd.read_csv('gbm_random_search_results.csv')
top = cv.sort_values('rank_test_score').head(10)
print(top[['param_subsample','param_n_estimators','param_min_samples_leaf','param_max_depth','param_learning_rate','mean_test_score','rank_test_score']])

# 2) load best model (if saved)
best = joblib.load('gbm_best_model.joblib')
print("Best model params:")
pprint(best.get_params())

    param_subsample  param_n_estimators  param_min_samples_leaf  \
9               0.8                 200                       1   
13              0.8                 100                       3   
10              0.6                 100                       3   
12              0.8                 100                       5   
7               0.8                 100                       3   
6               0.6                 100                       5   
8               0.6                 500                       3   
4               1.0                 200                      10   
15              1.0                 100                      10   
5               1.0                 200                       5   

    param_max_depth  param_learning_rate  mean_test_score  rank_test_score  
9                 2                 0.05       -10.058489                1  
13                3                 0.10       -10.105571                2  
10                2            

___

In [4]:
top.columns.unique()

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_subsample', 'param_n_estimators', 'param_min_samples_leaf',
       'param_max_depth', 'param_learning_rate', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score'],
      dtype='object')

In [3]:
top[['split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score']]

,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
9,-0.126330,-2.046628,-20.562349,-20.833878,-6.723258
13,-0.140045,-2.073272,-20.963669,-20.831506,-6.519363
10,-0.101690,-2.078260,-21.047754,-21.382583,-7.084388
12,-0.128702,-2.161694,-21.343423,-21.109383,-7.087615
7,-0.119499,-2.170665,-21.356039,-21.365984,-6.944857
6,-0.133227,-1.949627,-20.452409,-21.344772,-8.374607
8,-0.143259,-1.938623,-21.795890,-21.409125,-7.219376
4,-0.163340,-2.304249,-22.360361,-22.217388,-6.173743
15,-0.122820,-2.402143,-22.065559,-23.156947,-5.893453
5,-0.158640,-2.349923,-20.406453,-21.364260,-9.420626


In [9]:
print(best.get_params())

{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 2, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 0.8, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


____

# Diagnostics 

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('GLD_daily.csv')

In [6]:
df.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
5265,2025-10-23 00:00:00-04:00,381.459991,382.250000,378.649994,378.790009,13695500,0.0,0.0,0.0
5266,2025-10-24 00:00:00-04:00,378.510010,380.769989,376.809998,377.519989,13263800,0.0,0.0,0.0
5267,2025-10-27 00:00:00-04:00,371.130005,371.589996,365.339996,367.010010,23657600,0.0,0.0,0.0
5268,2025-10-28 00:00:00-04:00,361.809998,365.299988,360.119995,364.380005,19257600,0.0,0.0,0.0
5269,2025-10-29 00:00:00-04:00,369.649994,370.079987,361.359985,363.000000,18817400,0.0,0.0,0.0


In [3]:
signals = pd.read_csv('trade_ready_signals_output.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'trade_ready_signals_output.csv'